# Predicting the Errors of your model:

NannyML has released DLE, an algorithm able to predict the MAE and the MSE of your regression model, in the absence of the ground truth. This library includes both methods for classification and for regression

## Classification

They propose an algorithm called **“Confidence-Based Performance Estimation” (CBPE)**. This method allows to reliably estimate the performance of a model, by using predicted probabilities as the only input (i.e. when a ground truth is not available).

Let’s say your company asks you to predict the churn probability of each customer within the next month. So you develop a machine learning pipeline and, after validating it on hold-out data, you deploy it. The output of your pipeline is a table like this:

![](https://miro.medium.com/max/640/1*2rYYgl04VlwmwQVQOLrIbA.webp)

Typically, you set a small group of (randomly chosen) customers apart. The idea is to leave this group intact so that a month from now you will know what would happen if you did nothing to your customer base. As in clinical trials, this group is called the “control group”.

![](https://miro.medium.com/max/640/1*kFlbV9qCWYWcRBp5NmGtJA.webp)


One month later, you finally know which customers of the control group have churned. Thus, you can calculate any performance metric for your model on the control group: area under ROC curve, average precision, F1 score, and so on.

![](https://miro.medium.com/max/640/1*JJnlE4lBIVQvp2mpghcm_w.webp)

On the contrary, the algorithm proposed by NannyML allows you to get a reliable estimate of the model’s performance right now, without having to wait for the control group. **Additionally, these expected metrics are calculated on the whole customer base (not on the control group only).**


There are many benefits:

- Taking better decisions right now, allowing to make what-if analyses (for example: what are the expected precision/recall if I set the probability threshold to 50%? What if I set the threshold to 80%?).
- Having an early red flag about your model, so you can fix it before it’s too late. For example, if your credit scoring model is systematically failing, you don’t need to wait one year (and lose millions) before even realizing it.
- Having a realistic result about how your model will perform on the current data, since sometimes waiting for the control group is simply unfeasible, such as in long-term prediction (e.g. house mortgage defaults) or when getting labels is expensive.

### Warning, handle with care!

For the algorithm to produce reliable results, two pre-conditions must be met:

- The probabilities produced by the model must be properly calibrated.
- There must be no concept drift (however, data drift is not an issue).

Probability calibration is an intuitive concept: if you have a bunch of individuals whose predicted probability is, say, 20%, then around 20% of them should actually turn out to be positive. I won’t go into detail here, but there are techniques for calibrating your model “ex-post”.

### From probabilities to confusion matrix

To fix ideas, let’s take as an example 5 individuals, with different predicted probabilities: 10%, 25%, 40%, 70%, and 85%.

![](https://miro.medium.com/max/640/1*Nx7Dk7tjDMly_g_Uaf3IaQ.webp)


But, usually, probabilities are not used as they are. In fact, in the end, we are interested in finding out which individuals will be positive and which ones will be negative. To do that, we need to set a threshold: the observations above that threshold will be classified as positives, and the ones below the threshold will be classified as negatives.

For instance, if we set the threshold at 50%, the first 3 individuals will be classified as negatives (label 0), whereas the last 2 will be classified as positives (label 1).

![](https://miro.medium.com/max/640/1*AXcRF8A7RgkwUvTC3x61oA.webp)


Now let’s consider the last individual, whose predicted probability is 85%. Since the point is above the threshold, this individual is classified as positive. However, since our probabilities are calibrated, we can reasonably expect that:

- 85% of the times, this individual will actually turn out to be positive (“true positive”);
- 15% of the times, this individual will instead turn out to be negative (“false positive”).

Let’s now take the first individual. Its predicted probability is 10%, thus it is classified as negative. Following the same reasoning:

- 90% of the times, this individual will actually turn out to be negative (“true negative”);
- 10% of the times, this individual will instead turn out to be positive (“false negative”).

It’s easy to geometrically visualize these quantities for all the observations:

![](https://miro.medium.com/max/640/1*kC_uGzGDVzwAnwx3-PR9mw.webp)

Each colored segment represents the probability that an individual will be a true/false positive or a true/false negative.

Thus, if we sum the length of all the segments for each color we obtain respectively the expected number of true positives, false positives, false negatives, and true negatives.

Also, it’s clear from the figure that TP + FP + FN + TN = n, where n is the number of individuals. Note that this property is shared with “regular” confusion matrices.

The only difference is that, in our case, the values of TP, FP, FN, and TN are not integer values. And this makes sense, since they are expected values. In fact, in this example, this is what we get:

![](https://miro.medium.com/max/640/1*Z5KKik4821ctjoqOe2VMYQ.webp)

Note that, to obtain this expected confusion matrix, we only need to know the (calibrated) predicted probabilities and a threshold. The corresponding Python code of all we have seen so far is:

```python

pred = proba >= thres
    
tp = np.sum((pred == 1) * proba)
fp = np.sum((pred == 1) * (1 - proba))
fn = np.sum((pred == 0) * proba)
tn = np.sum((pred == 0) * (1 - proba))
```

Now that we know the expected confusion matrix, we can calculate many performance metrics, such as precision, recall, F1 score, average precision, area under the ROC curve, and so on.


Now, roc_auc_score is simply the area underlying this curve, which can be easily calculated: Scikit-learn also provides a function for computing directly the area under any curve. In our case, x-coordinates are given by False Positive Rates and y-coordinates are given by True Positive Rates, so:

```python

from sklearn.metrics import auc
fpr = [0.0, 0.06, 0.17, 0.39, 0.67, 1.0]
tpr = [0.0, 0.37, 0.67, 0.85, 0.96, 1.0]
roc_auc = auc(x=fpr, y=tpr)

```

## Regression

They have developed a method called **“Direct Loss Estimation” (DLE)** that allows estimating the performance (specifically Mean Absolute Error and Mean Square Error) of any regression model, when the ground truth is not available.

Basically, the idea is to directly predict the errors made by the model.

### Introducing the posterior distribution

We are used to models that return a single value for each observation, also called “point prediction”. However, we must keep in mind that, behind that point prediction, there is always a full distribution. If you like fancy statistical terms, you can call this the **“posterior distribution”.**

The cool thing about knowing the posterior distribution is that we can calculate anything we want: percentiles, mean, median… Anything.

However, most predictive models are designed to get a point prediction. Indeed, when given the 10,000 individuals above, your model will predict the same income for each of them. As you can guess, the models are usually designed to predict the mean of the posterior distribution.

### Why it doesn’t make sense to predict the error…

For each one of our individuals, the error is given by the difference between the point prediction and the true value. For example, if the income of a person is 65,000 $, the error made by the model is -15,000 $.

```python

error_distribution = point_prediction - posterior_distribution

# And if you take the mean error:
mean_error = np.mean(error_distribution)

```

At this point, it’s easy to see why it doesn’t make sense to predict the error. Because it would mean trying to predict something that is null by definition. ***

But didn’t we say that DLE is based exactly on predicting the error? So, what’s the catch?

*We are assuming that the model predicts the mean of the posterior distribution. This is not always the case. For example, if you use a loss function different from MSE, your errors may not be 0-centered. However, in that case, predicting the signed error would still be pointless: it would be like “correcting” the loss function that you chose in the first place.*


**… But it makes a lot of sense to predict the absolute error.**

*The point is that DLE does not predict the signed error, but the absolute error!*

It may seem like a small difference, but actually, it’s a whole different story. In fact, contrary to the signed error:

**The absolute error is a measure of how uncertain our point prediction is.**

```python

absolute_error_distribution = np.abs(point_prediction - posterior_distribution)

```
To sum up:

Trying to predict the signed error makes no sense, because we are trying to correct a model that we believe to be the best one.
Trying to predict the absolute (squared) error makes a lot of sense, because we are trying to quantify the uncertainty associated with the prediction.

### Describing DLE Algorithm

In practice, DLE trains a new model that learns the uncertainty associated with the predictions of the original model. In fact:

- The original model makes the point predictions (as always).
- The model introduced by DLE predicts the absolute (or squared) errors made by the main model. NannyML calls this one “nanny model” (for obvious reasons).

We can summarize the whole process in 4 steps.

As usual, everything starts with training the model on the training dataset.

![](https://miro.medium.com/max/640/1*Q-lK6-QlB_3AR5iseqzotA.webp)

Then, the model is used to make predictions on the test dataset. Once we have the predictions, we can also calculate the absolute errors as the absolute difference between target and prediction:

![](https://miro.medium.com/max/640/1*Bwwxc95cZAFPsZ48a5IGDg.webp)

At this point, a second model — called the nanny model — uses the original features and the predictions made by the first model to learn patterns about absolute errors.

![](https://miro.medium.com/max/640/1*qlHpcVAX5iowv-b8nnBMMA.webp)

Finally, when the model is used in production to make predictions, we can use the first model to predict the target variable and the nanny model to predict absolute errors.

![](https://miro.medium.com/max/640/1*3QeJdy4Cb38krAwlDMVJNA.webp)

### Predicting performance on a real dataset

Let’s see if DLE works on a real dataset.

We will use the Superconductor dataset, an open-source dataset from UCI. This dataset consists of 21,263 materials, with 81 features recorded regarding atomic mass, atomic radius, density, and others. The goal is to predict the critical temperature.

We will split the dataset into a training, a test, and a production dataset.

In [3]:
# load data and split into train, test, prod
import pandas as pd
import numpy as np

df = pd.read_csv('./data/train.csv').sample(frac=1, random_state=123)
df_train = df.iloc[:int(len(df)/5)]
df_test = df.iloc[int(len(df)/5):int(len(df)/5*2)]
df_prod = df.iloc[int(len(df)/5*2):]
X_train = df_train.drop("critical_temp", axis=1)
X_test = df_test.drop("critical_temp", axis=1)
X_prod = df_prod.drop("critical_temp", axis=1)
 
y_train = df_train["critical_temp"]
y_test = df_test["critical_temp"]
y_prod = df_prod["critical_temp"]

In [5]:
# train model
from lightgbm import LGBMRegressor
model = LGBMRegressor().fit(X=X_train,y=y_train)

In [8]:
# compute observed errors made by the model on test data
pred_test = pd.Series(model.predict(X_test),
  index=X_test.index).clip(0)
error_test = pred_test - y_test

Once we have the errors, we can train the nanny model. Note that the nanny model uses as features all the original features plus the predictions made by the first model.

In [9]:
# train model to predict the absolute error
model_abs_error = LGBMRegressor().fit(
  X=pd.concat([X_test, pred_test], axis=1),
  y=error_test.abs()
)

On the production dataset, we first use the main model to obtain the predictions. Then, we use the nanny model to get the predicted absolute errors.

In [10]:
# predict the absolute errors on production data
pred_prod = pd.Series(model.predict(X_prod), index=X_prod.index).clip(0)
pred_abs_error_prod = pd.Series(model_abs_error.predict(pd.concat([X_prod, pred_prod], axis=1)), index=X_prod.index)

In [12]:
# predict MAE on production set
pred_mae_prod = np.mean(pred_abs_error_prod)

### Testing DLE against data drift


The whole purpose of DLE is to predict the MAE (or MSE) of our model when we don’t have the ground truth. This is extremely useful in a real-world scenario when we want to know in advance if there is a deterioration in the performance of our model.

In order to simulate this scenario, we will split the production set into ten folds. **To reproduce data drift, we will not split the folds randomly, but we will divide them based on the predictions made by the model**. In this way, we make sure that the folds are sufficiently different from each other, and that the performances are reasonably different across the folders.

So, let’s compare the actual MAE on each folder with the MAE obtained as the mean of the absolute errors predicted by the nanny model.

![](https://miro.medium.com/max/640/1*K454plsK8Tr-xjLt7UT8ow.webp)

The results are impressive. The MAE predicted through the nanny model is practically the same as the actual MAE.

Let’s try with the MSE:

![](https://miro.medium.com/max/640/1*G_Cdel9_iHx1o3Rflh9Wow.webp)


### What happens if we try to estimate the error (rather than the absolute error)?


We have already seen above that trying to estimate the signed error makes no sense, from a theoretical point of view. But since data scientists prefer the practice over the theory, let’s try to do it anyway.

In other words, this means repeating the procedure above but, rather than training the nanny model on test absolute errors, training it on test signed errors.

Translated in Python, this means the following:

In [13]:
# train model to predict the error (which makes no sense)
model_error = LGBMRegressor().fit(
  X=pd.concat([X_test, pred_test], axis=1),
  y=error_test
)

Now, we are able to get a prediction of the signed error for each observation in the test set. If we take these predictions, take their absolute value, and then average them, this is a new estimate of the MAE.

Let’s see how it would perform on the same test folds as above:

![](https://miro.medium.com/max/640/1*-V4yIDr-b8sPON3USvBqJg.webp)


It’s evident how, using this new strategy, the predicted MAE systematically underestimates the actual MAE.

This is even worse if we try to do it for MSE:

![](https://miro.medium.com/max/640/1*ykjCv5_o3Q9ZzRW8jiY-2Q.webp)

This is just further proof that predicting the absolute error is completely different from predicting the signed error and then taking the absolute error.

### And if you don’t want to reinvent the wheel…

In this article, we have implemented DLE from scratch to show how it works under the hood. However, in real life, it’s often preferable to use libraries that are well maintained.

So, you may want to directly use NannyML, which has several native functionalities, such as fitting hyperparameters for you.

In [ ]:
# use DLE directly with NannyML
import nannyml as nml
estimator = nml.DLE(
    feature_column_names=df_train.columns.to_list(),
    y_pred='y_pred',
    y_true='critical_temp',
    metrics=['mae', 'mse'],
    chunk_number=10,
    tune_hyperparameters=False
)
estimator.fit(df_test)

In [ ]:
results = estimator.estimate(df_prod_drifted)
results.plot(metric=’mae’)

### When it won’t work

When using DLE to estimate performance, there are some assumptions that you should pay attention to. Indeed, if some of these assumptions are violated, the performance estimated by DLE may not be reliable anymore. These are the assumptions:

- There is no concept drift. If the relation between the features and the target variable changes in unforeseen ways, the error that you are trying to predict changes as well, and so the nanny model may fail.
- There is no covariate shift to previously unseen regions in the input space. Both the main model and the nanny model learn on the original features. If the features drift to values not seen during the training/validation phases, the models may make an uncorrect guess.
- The sample of data is large enough. Of course, both the training and the validation dataset need to be large enough for both the models to be robust.